# Preprocessing

In [ ]:
# Importing Libraries

import numpy as np
import tensorflow as tf
import datetime
import matplotlib.pyplot as plt
import torchvision
import pennylane as qml
import time
import datetime
import os
import csv
from sklearn.model_selection import train_test_split
from keras.utils.vis_utils import plot_model
from sklearn.datasets import load_digits
from pennylane import numpy as p_np
from pennylane.templates.state_preparations import MottonenStatePreparation, ArbitraryStatePreparation
from pennylane.templates.layers import StronglyEntanglingLayers

In [ ]:
# Load the digits

tf.keras.backend.set_floatx('float64')
digits = load_digits()
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target)
y_test_original = y_test

In [ ]:
num_classes = 10

In [ ]:
# Change Y values to categorical
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Compose the circuit

In [ ]:
num_qubits = 6
num_layers = 4
device_name = "default.qubit"

dev = qml.device(name = device_name, wires = num_qubits)

@qml.qnode(dev)
def circuit(weights, inputs=None):

    # Embedding 
    qml.AmplitudeEmbedding(inputs, wires = range(num_qubits), normalize=True)
    
    # Layers
    StronglyEntanglingLayers(weights, wires=range(num_qubits))
  
  # Measurement return
    return [qml.expval(qml.PauliZ(i)) for i in range(num_qubits)] 

# Create the QNN

In [ ]:
weight_shapes = {"weights": (num_layers, num_qubits, 3)}

# Model  
input_layer = tf.keras.layers.Input(shape=(2 ** num_qubits,), name = "input")
quantum_layer = qml.qnn.KerasLayer(circuit, weight_shapes, output_dim=num_qubits, name = "quantum_layer")(input_layer)
output_layer = tf.keras.layers.Dense(num_classes, activation='softmax', name = "output")(quantum_layer)

# Model creation
model = tf.keras.Model(inputs=input_layer, outputs=output_layer, name="mnist_quantum_model")

# Model compilation
model.compile(
  loss='categorical_crossentropy',
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.01) ,
  metrics=[tf.keras.metrics.CategoricalAccuracy()]
)

In [ ]:
plot_model(model, show_shapes=True, show_layer_names=True)

# Training

In [ ]:
num_epochs = 5                # Number of epochs

start = time.time()
train_history = model.fit(X_train, y_train, epochs=num_epochs, batch_size=8, shuffle=True)
end = time.time()
print(end - start)

# Evaluation

In [ ]:
start = time.time()
test_history = model.evaluate(X_test, y_test, batch_size=8)
end = time.time()
print(end - start)

In [ ]:
def tensor_to_image(tensor):
    tensor = tensor*255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor)>3:
        assert tensor.shape[0] == 1
        tensor = tensor[0]
    return PIL.Image.fromarray(tensor)

In [ ]:
predictions = model.predict(X_test)

In [ ]:
number = 11

bars = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')
y_pos = np.arange(len(bars))

# Create bars
plt.bar(y_pos, predictions[number])

# Create names on the x-axis
plt.xticks(y_pos, bars)

# Show graphic
plt.show()
print(y_test[number])

In [ ]:
data = X_test[number]

data = np.reshape(data, (-1, 8))
plt.imshow(data*255, cmap='gray')

# Prediction Pictures

In [ ]:
rows = 3
cols = 6

counter = 0;
for k in range(rows*cols):
    plt.subplot(rows, cols, k+1)
    #plt.xlabel('Pred: ' + str(np.argmax(predictions[k])))
    plt.xlabel('Pred: ' + str(np.argmax(predictions[k])) + '\nActual: ' + str(y_test_original[k]))
    plt.imshow(np.reshape(X_test[k], (-1, 8))*255, cmap='gray')
plt.tight_layout()